In [ ]:
# Step 1: Install OpenSCAD
!sudo apt-get install openscad

# Step 2: Upload .scad Files
from google.colab import files

# Function to upload the .scad file
def upload_scad_file():
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    print(f"Uploaded file: {filename}")
    return filename

# Step 3: Parse .scad File to Find Parameters
import re

def parse_scad_file(filename):
    """
    Parse the uploaded .scad file and identify the parameters.
    This function looks for lines in the SCAD file that define parameters.
    """
    with open(filename, "r") as file:
        scad_content = file.read()

    # Use regex to find all lines that define parameters (e.g., "param = value;")
    params = re.findall(r'(\w+)\s*=\s*([0-9.]+)', scad_content)
    parameters = {param[0]: float(param[1]) for param in params}
    return parameters, scad_content

# Step 4: Provide User Input Form to Change Parameters
from IPython.display import display, HTML
import ipywidgets as widgets

def display_param_form(parameters):
    """
    Displays the input form where the user can modify the parameter values.
    Each parameter will have a corresponding input field.
    """
    input_fields = []
    for param, value in parameters.items():
        label = widgets.Label(value=f"Change {param}:")
        textfield = widgets.FloatText(value=value, description=param)
        input_fields.append((label, textfield))

    # Display the input fields
    display(HTML("<h3>Customize Your Model</h3>"))
    for label, field in input_fields:
        display(label, field)

    submit_button = widgets.Button(description="Submit Changes")
    display(submit_button)

    return input_fields, submit_button

# Step 5: Update SCAD File Based on User Input
def update_scad_file(input_fields, scad_content, filename):
    """
    Update the .scad file with the new parameters entered by the user.
    """
    updated_params = {}
    for label, field in input_fields:
        updated_params[field.description] = field.value

    # Replace parameters in the original SCAD content
    for param, value in updated_params.items():
        scad_content = re.sub(rf'{param}\s*=\s*[0-9.]+', f'{param} = {value}', scad_content)

    updated_filename = f"updated_{filename}"
    with open(updated_filename, "w") as file:
        file.write(scad_content)
    print("Updated SCAD file saved.")

    # Print the content of the updated SCAD file to verify the changes
    with open(updated_filename, "r") as file:
        updated_content = file.read()
        print("Updated SCAD File Content:")
        print(updated_content)  # Print to verify the content

    return updated_filename  # Return the updated filename

# Step 6: Generate the STL File from Updated SCAD
import subprocess
import os

def generate_stl(updated_filename):
    """
    Generate an STL file from the updated SCAD file.
    """
    try:
        # Run the OpenSCAD command to generate STL
        result = subprocess.run(
            ["openscad", "-o", "updated_model.stl", updated_filename],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE
        )

        # Capture the stdout and stderr
        stdout, stderr = result.stdout.decode(), result.stderr.decode()

        # Print the output and errors
        print("OpenSCAD Output:", stdout)
        print("OpenSCAD Error:", stderr)

        # Check if STL file was generated
        if os.path.exists("updated_model.stl"):
            print("STL file generated: updated_model.stl")
        else:
            print("Failed to generate STL file.")

    except Exception as e:
        print(f"Error running OpenSCAD: {e}")

# Step 7: Download the STL File
def download_stl():
    """
    Provide the download link for the STL file.
    """
    if os.path.exists("updated_model.stl"):
        files.download("updated_model.stl")
    else:
        print("STL file not found, cannot download.")

# Putting Everything Together
def full_workflow():
    # Upload the SCAD file
    uploaded_file = upload_scad_file()

    # Parse the SCAD file to extract parameters
    parameters, scad_content = parse_scad_file(uploaded_file)

    # Show the form to allow user input for each parameter
    input_fields, submit_button = display_param_form(parameters)

    # Handle the form submission
    def on_submit(button):
        updated_filename = update_scad_file(input_fields, scad_content, uploaded_file)  # Update SCAD file
        generate_stl(updated_filename)  # Generate STL file
        download_stl()  # Provide STL file for download

    submit_button.on_click(on_submit)

# Run the full workflow
full_workflow()


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openscad is already the newest version (2021.01-4build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


Saving updated_model.scad to updated_model.scad
Uploaded file: updated_model.scad


Label(value='Change cube_size:')

FloatText(value=20.0, description='cube_size')

Label(value='Change thickness:')

...

FloatText(value=5.0, description='radius')

Button(description='Submit Changes', style=ButtonStyle())

Updated SCAD file saved.
Updated SCAD File Content:
// Example SCAD File for Testing

// Parameters
cube_size = 50.0; // Size of the cube
thickness = 15.0;  // Thickness of the outer shell
radius = 20.0;     // Radius of a hole in the center

// Define the Cube
difference() {
    // Outer Cube
    cube(cube_size);

    // Inner Hole (Cylinder)
    translate([cube_size / 2, cube_size / 2, 0])
    rotate([90, 0, 0])
    cylinder(h = cube_size + 2 * thickness, r = radius);
}


OpenSCAD Output: 
OpenSCAD Error: Geometries in cache: 4
Geometry cache size in bytes: 14144
CGAL Polyhedrons in cache: 1
CGAL cache size in bytes: 54352
Total rendering time: 0:00:00.048
   Top level object is a 3D object:
   Simple:        yes
   Vertices:       40
   Halfedges:     120
   Edges:          60
   Halffacets:     44
   Facets:         22
   Volumes:         2

STL file generated: updated_model.stl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>